# AI Agent for financial analyis

# AI Financial Analyst Agent

The AI Financial Analyst Agent is designed to analyze financial data for a given company. It utilizes the `company` object, which contains financial items and other relevant information. Additional context can be provided by attaching a list of files to enhance the analysis.

The agent is tasked with analyzing a specific `financialItem`. Using all the available data, it performs the following:

1. **Analysis**: Generates a concise text summary of the financial item.
2. **Estimation**: Provides an estimate for the next 4 periods.
3. **Conviction Score**: Outputs a confidence score for the analysis and estimates.

This tool is highly versatile and can adapt to various financial contexts by leveraging the provided data and additional files.

In [6]:
from IPython.display import Markdown, display
from PyFinModeler import (Company, BorsdataCollector) 
import os

api_key = os.environ.get("BORSDATA_API_KEY")
open_ai_api_key = os.environ.get("OPENAI_API_KEY")
if not open_ai_api_key: 
    raise ValueError("OpenAI API Key not found. Please set OPENAI_API_KEY.")
if not api_key: 
    raise ValueError("API Key not found. Please set BORSDATA_API_KEY.")

collector = BorsdataCollector(api_key=api_key)
company = collector.fetch_company("Evolution", report_type="quarter")

display(Markdown(f"**Company Name:** {company.name}"))  # Output: Company A
display(Markdown(f"**Description:** {company.description}"))  # Output: The English description of Company A

**Company Name:** Evolution

**Description:** Evolution operates in the gaming industry and focuses on the development and delivery of live casino solutions for online gaming companies. The company's platforms offer realistic gaming experiences with live dealers. The business is global with the largest presence in Europe, North America and Asia. Evolution was founded in 2006 and is headquartered in Stockholm, Sweden.

In [7]:
income_statement = company.get_statement_as_dataframe("income_statement")
income_statement

,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3,2016Q4,2017Q1,2017Q2,2017Q3,...,2022Q4,2023Q1,2023Q2,2023Q3,2023Q4,2024Q1,2024Q2,2024Q3,2024Q4,2025Q1
Item,,,,,,,,,,,,,,,,,,,,,
revenues,164.789453,182.258404,205.662732,229.044493,254.941052,281.475746,328.80476,379.246622,407.307677,439.798233,...,4548.780736,4835.843390,5192.202578,5227.698251,5293.527470,5771.149414,5773.656483,6551.249912,7162.835843,5659.014672
gross_Income,164.789453,182.258404,205.662732,229.044493,254.941052,281.475746,328.80476,379.246622,407.307677,439.798233,...,4548.780736,4835.843390,5192.202578,5227.698251,5293.527470,5771.149414,5773.656483,5876.357882,6115.674936,5659.014672
operating_Income,58.490938,58.832257,68.896740,76.369719,78.035127,78.864481,96.82506,133.349794,153.474765,175.043354,...,2790.844653,3056.131804,3313.910126,3316.231354,3370.033926,3586.299838,3533.274263,4290.107042,4784.373341,3296.214405
profit_Before_Tax,58.564744,58.738724,68.869218,76.212675,77.556038,78.372963,95.72336,132.528003,153.031726,174.581321,...,2684.414704,3040.090152,3331.485274,3378.135602,3375.034439,3653.937643,3610.406252,4278.691014,4872.709786,3282.679107
profit_To_Equity_Holders,54.413179,54.155607,63.750126,70.577556,71.553336,71.655556,86.67984,121.758729,140.780712,161.105341,...,2494.997526,2827.270895,3108.776481,3150.187068,3150.200683,3097.700997,3056.468651,3717.970576,4320.065714,2766.469347
net_Sales,164.789453,182.258404,205.662732,228.462505,252.573789,281.254081,328.06710,379.246622,406.517910,439.547965,...,4548.780736,4835.843390,5192.202578,5227.698251,5293.527470,5771.149414,5773.656483,5876.357882,6115.674936,5659.014672


### Setting up the agent

In [8]:
# Import necessary modules
from PyFinModeler import AnalystAgentContext, run_analyst_agent
import pprint

# Assuming `company` is already defined and has a `to_dict()` method
company_json = company.to_dict()

# Define the context for the agent
context: AnalystAgentContext = {
    "company": company_json,
    "transcripts": [
        open("data/evo_transcript_2024Q4.txt").read(),
        open("data/evo_transcript_2025Q1.txt").read()
    ],
    "target_item": "revenues"
}

In [9]:
ASSISTANT_ID = "asst_1JOObU2tF9VsV0tTokWME3Fp"

# Run the agent directly in a notebook cell
output = await run_analyst_agent(context, ASSISTANT_ID)


In [10]:
pprint.pp({
    "analysis":output["analysis"],
    "next_4_qs":output["next_4_qs"],
    "conviction_score": output["conviction_score"],
    "reasoning": output["reasoning"],
    })

{'analysis': "Evolution's revenues have shown a strong upward trajectory since "
             '2015, reflecting significant growth driven by strategic '
             'expansions and high demand in the gaming industry. Revenues '
             'escalated from EUR 164 million in 2015 to EUR 5,293 million in '
             '2023, marking substantial annual increases with a notable peak '
             'in 2020 coinciding with an increased global shift towards online '
             'entertainment due to the pandemic. Additionally, expansion into '
             'North American and Asian markets and innovative game releases '
             'have contributed to this robust growth. However, anomalies do '
             'exist, such as the flat revenue growth observed in 2023 despite '
             'historical trends, mainly due to cyberattacks in Asia and '
             'regulatory changes in some European markets as elaborated in '
             'earnings calls. Key management comments highlighted